# Projet 4

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import multiprocessing

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [2]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    if url == "https://kim.fspot.org/cours/page2.html":
        global_offers = soup.find_all(class_="pure-u-1 pure-u-md-1-3")
    else:
        global_offers = soup.find_all(class_="pure-u-1 pure-u-md-1-4")
    for offer in global_offers:
        name = offer.find("h2").text
        price = offer.find('span').text.split()[0]
        storage = offer.find_all('li')[3].text.split()[0]
        database = offer.find_all('li')[4].text.split()[0]
        prices[name] = {'price':price, 'storage':storage, 'databases':int(database)}
    return prices


Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [3]:
def extract_beer_infos(url):

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    beer_details = soup.find_all(class_="small-12 content-column")[0]
    name = beer_details.find_all(class_="product-detail-info-title")[0].find("h1").text
    note = beer_details.find(class_="stars")['data-percent']
    price = soup.find(class_="price").text.split()[0].replace(',', '.')
    volume = beer_details.find(class_="small-6 medium-9 columns js-beer-volume").text.split()[0]
    infos = {
        'name': name,
        'note': int(note),
        'price': float(price),
        'volume': int(volume),
    }
    return infos

Cette URL retourne un JSON avec une liste de bières :

In [4]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [5]:
import sub_func

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    root_site = 'https://www.beerwulf.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    beer_json = json.loads(soup.text)
    #print(beer_json['items'])
    for beer in beer_json['items']:
        beer_pages.append(root_site+beer['contentReference'])

    # Sequential version (slow):
    #beers = []
    #for beer_page in beer_pages:
    #    beers.append(extract_beer_infos(beer_page))

    # Parallel version (faster):
    p = multiprocessing.Pool(2)
    beers = p.map(sub_func.extract_beer_infos, beer_pages)
    return beers


In [6]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.524s

OK
